In [6]:
from datetime import timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

import folium
import folium.plugins as plugins
import numpy as np

# read the first 100 signal values... 100 is just a sample, as 100 alone will create a 3 MB+ html file. Loading the entire
# dataset will be slow!

# dataframe which expects a T x N csv file (see example)
df = pd.read_csv('final_10min.csv', nrows=100)

# interpolating to make sure that there are no nans; if the file is generated by the previous scripts, it should not
# contain any
df.interpolate(inplace=True, axis=1)

print("{} nans left in the df".format(df.isna().sum().sum()))

# gps data (see example) N x N
df_2 = pd.read_csv('gps_10min.csv')

# normalizing the signals' df to make sure that the values are on the same scale for the heatmap. Skipping this step
# will result in a map where most of the times the nodes are appearing to be "off".
df = (df - df.mean()) / (df.std() ** 1.2)

# clean up - be sure to check the example if using your own data
df_2.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

0 nans left in the df


In [3]:
# Outer list:
list_df_gps = df_2.columns.values.tolist()
OuterList = []
for index, row in df.iterrows():
    Mid_List = []
    for j in list_df_gps:
        Time_List = df_2[j].values.tolist()
#         print(type(j))
#         print(row[0])
        Time_List.append(row[j])
        Mid_List.append(Time_List)
    OuterList.append(Mid_List)

In [4]:
# init the map
m = folium.Map([47.36179377478453, 8.572671632662107], tiles='stamentoner', zoom_start=12)

list_of_id = df_2.values
list_of_id = list(zip(list_of_id[0], list_of_id[1]))

### Folium's heatmap requires a list of list of list, where the first list is the number of time steps (100 for example, will represent the first 100 time steps), the second list is the graph's nodes (so, length of N), and the last list is 3 where the first and second elements are GPS coord. and the third is the signal value (heatmap's value).

In [5]:
for j in list_of_id:
    folium.Circle(j, popup='<strong>Location One</strong>', radius=20).add_to(m)

index = [i for i in range(len(OuterList))]

hm = plugins.HeatMapWithTime(OuterList, auto_play=True, control=True)

hm.add_to(m)

# save to local storage as index.html
m.save('index.html')

## Note about this visualization!

There's a bug as of Oct. 2020 with folium's heatmapwithtime class. It's very well explained here:

https://github.com/python-visualization/folium/issues/1221

However, if the above was not available, simply replace:

"https://rawcdn.githack.com/socib/Leaflet.TimeDimension/master/dist/leaflet.timedimension.min.js"

in the index.html file (it is referenced as a script tag, you may opne index.html via sublime or any other text editor, and find it using ctrl+f) by:

"https://cdn.jsdelivr.net/npm/leaflet-timedimension@1.1.0/dist/leaflet.timedimension.min.js"

Be sure to save changes, and reload the page.